In [ ]:
import re
import os
path = 'logs'
strn = 'iou'
names=[]
for dirpath, dirnames, filenames in os.walk(path):
    # print (dirpath, '\t', dirnames, '\t', filenames, '\n')
    for filepath in filenames:
        model_name=filepath
        str1 = re.compile(strn+'(.*?).pt')
        match_obj = re.findall(str1,model_name)
        # print (match_obj)
        if match_obj:
            names.append(path +'\\'+ model_name)




In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Dense_Layer(nn.Module):
    def __init__(self, c1, growth, e = 0.5): 
        super(Dense_Layer, self).__init__()
        self.act = nn.ReLU(inplace=True)
        self.bn1 = nn.BatchNorm2d(c1)
        self.conv1 = nn.Conv2d(c1, int(growth * e), kernel_size = 1, stride= 1, bias=False)
        self.bn2 = nn.BatchNorm2d(int(growth * e))
        self.conv2 = nn.Conv2d(int(growth * e), growth, 3, 1, 1, bias=False)

    def forward(self, x_in):
        x = self.act(self.conv1(self.bn1(x_in)))
        x = self.act(self.conv2(self.bn2(x)))
        return torch.cat((x_in, x), 1)
    
class _Dense_Block(nn.Module):
    def __init__(self, c1, c2, n, e = 0.5):
        super(_Dense_Block, self).__init__()
        self.n = n
        self.layers = [Dense_Layer(c1 + c2 * i, c2, e) for i in range(n)]
    def forward(self, x):
        for i in range(self.n):
            x = self.layers[i](x)
        return x

class Transition(nn.Module): #size -> size/2
    def __init__(self, c1, c2, size_to_half = True):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(c1)
        self.conv = nn.Conv2d(c1, c2, 1, 1)
        self.act = nn.ReLU(True)
        self.pool = nn.AvgPool2d(2,2)
        self.flag = size_to_half
       
    def forward(self, x):
        x = self.act(self.conv(self.bn(x)))
        return self.pool(x) if self.flag else x

class Dense_Block(nn.Module): #size -> size/2
    def __init__(self, ch_in, ch_out, n, e = 0.5, growth = 32, size_to_half = True):
        super(Dense_Block, self).__init__()
        self.dense = _Dense_Block(n = n, c1 = ch_in, c2 = growth)
        self.tran = Transition(ch_in + growth * n, ch_out, size_to_half)
        self.x = nn.ConvTranspose2d(ch_in, ch_out, kernel_size=9, stride=1, padding=4, output_padding=0
                                      )
        self.x1 = nn.ConvTranspose2d(ch_in, ch_out, kernel_size=4, stride=2, padding=2, output_padding=1
                                      )

    def forward(self, x):
        # return self.x1(self.x(self.tran(self.dense(x))))
        return self.x(self.tran(self.dense(x)))


x = torch.rand((1, 32, 112, 112))
net = Dense_Block(32, 32, 3)
output = net(x)
print(output.shape)

In [ ]:
import torch
from inference.models.grconvnet3 import GenerativeResnet
check = torch.load('trained_models\jacquard-rgbd-grconvnet3-drop0-ch32\epoch_35_iou_0.92.pt')
params = check.state_dict()
new_model = GenerativeResnet(channel_size=32)
new_model.load_state_dict(params,strict=False)
for name, param in new_model.named_parameters():
    if name.startswith('conv'):
        param.requires_grad = False
l = [(i, j) for i, j in new_model.named_parameters()]
l

In [ ]:
import logging

file_handler = logging.FileHandler('evaluate_results.txt')

logging.getLogger().addHandler(file_handler)

logging.info('This message will be printed and saved in log.txt')


In [ ]:
import os
kn = 'logs\\230724_1040_oringin'
os.path.dirname(kn)

In [ ]:
import json
vars(json)

In [ ]:
import glob
import os
# glob.glob(r'./*.py')
file_path = "D:\\1_PRP\Cornell"
grasp_files = glob.glob(os.path.join(file_path, '*', 'pcd*cpos.txt'))  # 取出点云数据和正向抓取矩形标签
grasp_files.sort()
ds_rotate = 0.5
length = len(grasp_files)
grasp_files = grasp_files[int(length * ds_rotate):] + grasp_files[:int(length * ds_rotate)]



In [ ]:
def iou(self, gr, angle_threshold=np.pi / 6):
        """
        Compute IoU with another grasping rectangle
        :param gr: GraspingRectangle to compare
        :param angle_threshold: Maximum angle difference between GraspRectangles
        :return: IoU between Grasp Rectangles
        """
        if abs((self.angle - gr.angle + np.pi / 2) % np.pi - np.pi / 2) > angle_threshold:
            return 0

        rr1, cc1 = self.polygon_coords()
        rr2, cc2 = polygon(gr.points[:, 0], gr.points[:, 1])

        try:
            r_max = max(rr1.max(), rr2.max()) + 1
            c_max = max(cc1.max(), cc2.max()) + 1
        except:
            return 0

        canvas = np.zeros((r_max, c_max))
        canvas[rr1, cc1] += 1
        canvas[rr2, cc2] += 1
        union = np.sum(canvas > 0)
        if union == 0:
            return 0
        intersection = np.sum(canvas == 2)
        return intersection / union


In [ ]:
#  此函数将网络输出的三幅图转换成五维抓取表示
def detect_grasps(q_img, ang_img, width_img=None, no_grasps=1):  # no_grasp: 返回的最大抓取数量
    """
    Detect grasps in a network output.                                此函数将网络输出的三幅图转换成抓取矩形
    :param q_img: Q image network output
    :param ang_img: Angle image network output
    :param width_img: (optional) Width image network output
    :param no_grasps: Max number of grasps to return
    :return: list of Grasps
    """
    # 获得质量图中的局部最大值， 每个极大值对应一个抓取点，两个极大值点之间距离必须大于20.
    local_max = peak_local_max(q_img, min_distance=20, threshold_abs=0.2, num_peaks=no_grasps)

    grasps = []
    for grasp_point_array in local_max:
        grasp_point = tuple(grasp_point_array)

        grasp_angle = ang_img[grasp_point]  # 通过抓取点坐标在角度图上获取抓取角度

        g = Grasp(grasp_point, grasp_angle)
        if width_img is not None:
            g.length = width_img[grasp_point]
            g.width = g.length / 2

        grasps.append(g)

    return grasps

In [3]:
import torch
import torch.nn as nn
from inference.models.grasp_model import GraspModel, ResidualBlock, CoordAtt, CARes, C3, C3_CA, Dense_Block, GSConv, GSBottleneck, VoVGSCSP,\
    Trans,CA_GS_IRes,GS_IRes,MultiModal

class Conv(nn.Module):
    # ConvBNSiLU Module
    def __init__(self, c1, c2, k=1, s=1, p=0, g=1):
        super(Conv, self).__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, p, g)
        self.bn = nn.BatchNorm2d(c2)
        self.act = nn.ReLU()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))
    
class Conv_Mish(nn.Module):
    # Standard convolution
    def __init__(self, c1, c2, k=1, s=1, p=0, g=1, act=True): 
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, p, groups=g, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = nn.Mish()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))
    
x = torch.rand((1,4,112,112))
net1 = Conv(4, 4, 7, 1, 3, 4)
net2 = Conv_Mish(4, 4, 7, 1, 3, 4)
output1 = net1(x)
output2 = net2(x)
print(output1.shape)
print(output2.shape)

torch.Size([1, 4, 94, 94])
torch.Size([1, 4, 112, 112])


: 